# Offline Step: Generation of Training Artifacts

In [8]:
%pip install --upgrade pip
%pip install torch-ort torchvision

Note: you may need to restart the kernel to use updated packages.
  Using cached torch_ort-1.19.2-py3-none-any.whl.metadata (1.7 kB)
INFO: pip is looking at multiple versions of torch-ort to determine which version is compatible with other requirements. This could take a while.
  Using cached torch_ort-1.19.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached torch_ort-1.18.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached torch_ort-1.17.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached torch_ort-1.16.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached torch_ort-1.15.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached torch_ort-1.14.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached torch_ort-1.13.1-py3-none-any.whl.metadata (1.7 kB)
INFO: pip is still looking at multiple versions of torch-ort to determine which version is compatible with other requirements. This could take a while.
  Using cached torch_ort-1.12.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached torch_ort-1.11.1-p

In [9]:
# largely borrowed from https://www.youtube.com/watch?v=u7YCaiHOC9o

import torch
import os
import torchvision

os.makedirs("training_artifacts", exist_ok=True)

model = torchvision.models.mobilenet_v2(
    weights=torchvision.models.MobileNet_V2_Weights.IMAGENET1K_V2
)


# imagenet had 1000 classes, bloodMNIST only has 8
model.classifier[1] = torch.nn.Linear(1280, 8)

# convert the model to ONNX
model_name = "mobilenetv2"
torch.onnx.export(model, torch.randn(1, 3, 224, 224),
                  f"training_artifacts/{model_name}.onnx",
                  input_names=["input"],
                  output_names=["output"],
                  dynamic_axes={"input": {0: "batch"}, "output": {0: "batch"}},
                  export_params=True,
                  do_constant_folding=False,
                  training=torch.onnx.TrainingMode.TRAINING)

/Users/aopwtf/Desktop/decai-fe/venv/lib/python3.11/site-packages/torch/onnx/utils.py:491: FutureWarning: Setting `training` to something other than default is deprecated. The option will be removed in a future release. Please set the training mode before exporting the model.
  warnings.warn(
/Users/aopwtf/Desktop/decai-fe/venv/lib/python3.11/site-packages/torch/onnx/utils.py:616: UserWarning: ONNX Preprocess - Removing mutation from node aten::add_ on block input: 'features.0.1.num_batches_tracked'. This changes graph semantics. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/jit/passes/onnx/remove_inplace_ops_for_onnx.cpp:354.)
  _C._jit_pass_onnx_remove_inplace_ops_for_onnx(graph, module)
/Users/aopwtf/Desktop/decai-fe/venv/lib/python3.11/site-packages/torch/onnx/utils.py:616: UserWarning: ONNX Preprocess - Removing mutation from node aten::add_ on block input: 'features.1.conv.0.1.num_batches_tracked'. This changes graph semantics. (Triggered internall

Now we have a model converted to ONNX format, need to generate the training artifacts that we will use later:

In [10]:
import onnx
from onnxruntime.training import artifacts

# get the onnx model
onnx_model = onnx.load(f"training_artifacts/{model_name}.onnx")

ModuleNotFoundError: No module named 'onnxruntime.training'